In [ ]:
import axios from "axios";

// 1. Khởi tạo instance
const httpRequest = axios.create({
    baseURL: "https://api01.f8team.dev/api",
    timeout: 10000,
    headers: {
        "Content-Type": "application/json",
    },
});

axios.interceptors.response.use((response) => {
    return response.data;
});

// ==========================================================
// 2. BIẾN TRẠNG THÁI & HÀNG ĐỢI (QUEUE)
// ==========================================================
let isRefreshing = false;
let failedQueue = [];

/**
 * Thêm request lỗi vào hàng đợi
 */
const addToQueue = (resolve, reject) => {
    failedQueue.push({ resolve, reject });
};

/**
 * Xử lý hàng đợi sau khi Refresh Token xong
 * @param {Error} error - Lỗi nếu refresh thất bại
 */
const processQueue = (error) => {
    failedQueue.forEach((prom) => {
        if (error) {
            prom.reject(error);
        } else {
            prom.resolve(); // Chỉ cần resolve để báo hiệu "Đã refresh xong"
        }
    });
    failedQueue = []; // Xóa sạch hàng đợi
};

// ==========================================================
// 3. LOGIC GỌI API REFRESH TOKEN
// ==========================================================
const refreshAccessToken = async () => {
    try {
        const refreshToken = localStorage.getItem("refreshToken");
        
        // Gọi API Refresh (Dùng axios instance thường để tránh lặp interceptor vô tận)
        const response = await axios.post(
            "https://api01.f8team.dev/api/auth/refresh-token",
            { refresh_token: refreshToken }
        );

        const { access_token, refresh_token } = response.data;

        // Lưu ngay token mới vào Storage
        localStorage.setItem("accessToken", access_token);
        localStorage.setItem("refreshToken", refresh_token);

        return access_token;
    } catch (error) {
        // Nếu Refresh cũng lỗi -> Xóa sạch, bắt đăng nhập lại
        localStorage.removeItem("accessToken");
        localStorage.removeItem("refreshToken");
        throw error;
    }
};

// ==========================================================
// 4. REQUEST INTERCEPTOR (Gắn Token)
// ==========================================================
httpRequest.interceptors.request.use(
    (config) => {
        // Luôn lấy token mới nhất từ localStorage
        const token = localStorage.getItem("accessToken");
        if (token) {
            config.headers.Authorization = `Bearer ${token}`;
        }
        return config;
    },
    (error) => Promise.reject(error)
);

// ==========================================================
// 5. RESPONSE INTERCEPTOR (Xử lý lỗi 401 & Refresh)
// ==========================================================
httpRequest.interceptors.response.use(
    (response) => response.data, // Trả về data gọn gàng
    async (error) => {
        const originalRequest = error.config;

        // Nếu lỗi không phải 401 hoặc đã retry rồi -> Bỏ qua
        if (error.response?.status !== 401 || originalRequest._retry) {
            return Promise.reject(error);
        }

        // Đánh dấu để không lặp vô tận
        originalRequest._retry = true;

        // --- TRƯỜNG HỢP 1: ĐANG CÓ TIẾN TRÌNH REFRESH (Followers) ---
        // Các request đến sau sẽ phải chờ
        if (isRefreshing) {
            return new Promise((resolve, reject) => {
                addToQueue(resolve, reject);
            })
                .then(() => {
                    // Khi được resolve, gọi lại request cũ
                    // Request Interceptor sẽ tự động lấy token mới trong localStorage
                    return httpRequest(originalRequest);
                })
                .catch((err) => Promise.reject(err));
        }

        // --- TRƯỜNG HỢP 2: REQUEST ĐẦU TIÊN GẶP LỖI (Leader) ---
        isRefreshing = true;

        try {
            // 1. Gọi API lấy token mới và lưu vào localStorage
            await refreshAccessToken();

            // 2. Refresh thành công -> Báo cho các followers trong hàng đợi chạy lại
            processQueue(null);

            // 3. Gọi lại request của chính mình (Leader)
            // Lưu ý: Không cần set header thủ công, httpRequest sẽ chạy lại qua Request Interceptor
            return httpRequest(originalRequest);
        } catch (refreshError) {
            // Refresh thất bại -> Báo lỗi cho toàn bộ hàng đợi
            processQueue(refreshError);
            
            // Xử lý logout hoặc đá về trang login (nếu cần)
            // window.location.href = '/login'; 
            
            return Promise.reject(refreshError);
        } finally {
            // Luôn tắt cờ refresh dù thành công hay thất bại
            isRefreshing = false;
        }
    }
);

export default httpRequest;